# [IAPR][iapr]: Final project - Chocolate Recognition


**Moodle group ID:** *39*  
**Kaggle challenge:** *`Classic`*
**Kaggle team name (exact):** "*xx*"  

**Author 1 (SCIPER):** *Léo Bruneau (xxxxx)*  
**Author 2 (SCIPER):** *Louis Pivron (xxxxx)*  
**Author 3 (SCIPER):** *Huckleberry Thums (xxxxx)*  

**Due date:** 21.05.2025 (11:59 pm)


## Key Submission Guidelines:
- **Before submitting your notebook, <span style="color:red;">rerun</span> it from scratch!** Go to: `Kernel` > `Restart & Run All`
- **Only groups of three will be accepted**, except in exceptional circumstances.


[iapr]: https://github.com/LTS5/iapr2025

---

## Justification of Design Choices

### Features

In our project we use a variety of features. Since we had trouble dealing with noisy backgrounds, our method does not rely on segmentations of the chocolates. So, the features we compute do not include any shape or contour based features. Instead, we use a combination of texture features, color features, and some basic statistics. Below we list the features we compute:

Color based features: color statistics such as means and standard deviations in multiple color spaces (RGB, LAB) and color histograms in these same color spaces.
Texture based features: Local Binary Patterns (LBP), Haralick GLCM features, and Gabor energy.

Note that these features were computed on subdivisions of the segmentated reference chocolates and on the extracted patches of the train/test images.

### Sliding Window

One of the core parts of our pipeline is the sliding window detector. We came about this idea after having tried multiple classical object detection and segmentation methods. For example, we tried using region growing and contour detection, but these methods did not perform well on images with non-uniform backgrounds. Due to the clutter and the variety of backgrounds, it was difficult to foresee using contour or region based methods.

Using a sliding window is much more robust to these variations. The idea is to take a window of a fixed size and slide it over the image with a fixed stride. For each position of the window, we compute the histogram of the pixels inside the window. This histogram is then compared to a set of histograms that we have pre-computed for each reference chocolate. The best-matching histogram is then used to determine the likelihood of the window containing a chocolate. This is done by computing the Bhattacharyya distance between the histogram of the window and the histograms of the reference chocolates. The Bhattacharyya distance is a measure of similarity between two probability distributions.

This procedure leaves us with a heatmap showing the likelihood of each pixel being part of a chocolate. By thresholding, we can obtain a binary mask of the detections. The detections are then in the form of 'blobs' in the heatmap. To detect these blobs we use edge detection through the Laplacian of Gaussian (LoG) method. This method has input parameters that allow us to control the size of the blobs we want to detect, and how many by specifying a threshold.

After the blobs have been detected, we can use the bounding boxes of these blobs to crop the original image and obtain the chocolate candidates. From these patches, we can then compute the features we want to use for classification. 

### Classification

In [ ]:
# why we used a certain classifier
# use of training set (fully labaled, by hand): supervised classification
# optimization

## Technical Description

In [ ]:
# > A schematic of your architecture is sufficient.

### Data Preprocessing (general)

In [5]:
# parler ici du preprocessing fait sur les references et le training / test set (par exemple le downsampling)
# ...

### Reference Image Processing

In [ ]:
# Preprocessing specifique aux images de reference
# Reference histograms used for later
# Segmentation with masks and stuff

### Sliding Window

In [ ]:
# How the sliding window works
# Heatmap generation

### Blob Detection

In [ ]:
# LoG method for blob detection, thresholding 

### Patch Extraction

In [ ]:
# self-explanatory: patches extracted from the detected blobs
# resizing

### Features

In [ ]:
# How we extract the features and details of which ones
# Talk about PCA

### Classification

In [ ]:
# how labeling was done
# SVM
# optimization

## Quantitative and Qualitative Analysis

### Quantitative Results

In [ ]:
# > For the quantitative analysis, your Kaggle results, along with some intermediate results obtained throughout the project, should be sufficient. 

### Qualitative Results

In [ ]:
# > For qualitative analysis, we are looking for an interpretation of how your model works. Your model is not counting chocolates "magically"—it likely segments them internally and uses that information to compute useful descriptors.
# > We expect you to show some examples of this internal segmentation (e.g., binary masks), and to demonstrate that the model can extract meaningful features.
# > A helpful suggestion: you can extract the features and visualize them using a 2D PCA or t-SNE plot to assess whether the model learns discriminative representations.

## TA comments from the forum

In [ ]:
# Mais ducoup on peux upload des images des resultats intermediraires?
# Par exemple la visualization de la classification, de la segmentation, ...
# Ou on doit upload le code pour generer ces resultats?

# Reponse sur le forum:
# > Hello, Yes you can include the code that generates the figures in your notebook

# Sauvegarder le classifier, scaler, etc. :
# > As we will have to run your code in main.py, you can't use joblib. If you have a sklearn model, you can use pickle to save it. 

# > It is better to have a fully narrative description.
# Donc en gros ne pas mettre de code dans le rapport

# > Label for L1000780 is wrong: there are two jelly milk items instead of one jelly milk and one jelly black.